In [23]:
from selenium import webdriver
import time
import os
import pandas as pd
from trello import TrelloClient

import config

# コンテストの内容を markdown 形式で ipynb に出力

In [6]:
def get_task_urls(driver, contest_url):
    
    driver.get(contest_url)
    rows = driver.find_elements_by_xpath('//table/tbody//tr')
    task_urls = []
    for row in rows:
        task_url = row.find_element_by_xpath('td/a').get_attribute('href')
        task_urls.append(task_url)
    return task_urls

def create_ipynb(driver, task_url, path_ipynb):
    
    driver.get(task_url)
    html = driver.page_source
    html = html.split('<div class="col-sm-12">\n\t\t')[-1]
    html = html.split('\n<span class="lang-en"')[0]
    html = html.split('<div class="a2a_kit a2a_kit_size_20 a2a_default_style pull-right"')[0]

    html = html.replace('class="mjx-chtml MathJax_CHTML MJXc-processed" tabindex="0"',
                        'class="mjx-chtml MathJax_CHTML MJXc-processed" tabindex="0" style="display: none;"')
    html = html.replace('<span class="btn-copy btn-pre" tabindex="0" data-toggle="tooltip" data-trigger="manual" title=""',
                        '<span class="btn-copy btn-pre" tabindex="0" data-toggle="tooltip" data-trigger="manual" title="" style="display: none;"')
    html = html.replace('<pre class="prettyprint linenums source-code prettyprinted" style="">',
                        '<pre class="prettyprint linenums source-code prettyprinted" style="display: none;">')
    html = html.replace('"', "'")
    html = html.replace('\t', '')
    html = html.replace('\\', '\\\\')
    html = html.replace('\n', ' \\n",\n"')
    html = '"' + html + '\\n"'

    head = '{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "metadata": {},\n   "source": [\n'
    tail = '   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": null,\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "from ipywidgets import Textarea\\n",\n    "import sys\\n",\n    "sys.path.append(\'../../..\')\\n",\n    "\\n",\n    "from utils.multi_line_input import multi_line_input\\n",\n    "text_area = Textarea()\\n",\n    "input = multi_line_input()\\n",\n    "\\n",\n    "display(text_area)"\n   ]\n  }\n ],\n "metadata": {\n  "kernelspec": {\n   "display_name": "Python 3",\n   "language": "python",\n   "name": "python3"\n  },\n  "language_info": {\n   "codemirror_mode": {\n    "name": "ipython",\n    "version": 3\n   },\n   "file_extension": ".py",\n   "mimetype": "text/x-python",\n   "name": "python",\n   "nbconvert_exporter": "python",\n   "pygments_lexer": "ipython3",\n   "version": "3.7.1"\n  }\n },\n "nbformat": 4,\n "nbformat_minor": 2\n}\n'
    with open(path_ipynb, 'w', newline='\n', encoding='utf-8') as f:
        f.write(head + html + tail)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(config.path_to_chromedriver, chrome_options=options)
driver.set_window_size(1920, 1080)

alphabet = 'abcdefghijklmnopqrstuvwxyz'
for i in range(1, 140):
    time.sleep(1)
    contest_url = f'https://atcoder.jp/contests/abc{i:03}/tasks'
    task_urls = get_task_urls(driver, contest_url)
    os.mkdir(f'contests_create_new/ABC/ABC{i:03}')
    
    for j, task_url in enumerate(task_urls):
        time.sleep(1)
        path_ipynb = f'contests_create_new/ABC/ABC{i:03}/ABC{i:03}_{alphabet[j]}.ipynb'
        create_ipynb(driver, task_url, path_ipynb)
        
driver.close()

# Create New Trello Cards

In [24]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(config.path_to_chromedriver, chrome_options=options)
driver.set_window_size(1920, 1080)

di = {'contest': [],
      'url': [],
      'alphabet': [],
      'task': []}

for i in range(1, 140):
    time.sleep(1)
    contest_url = f'https://atcoder.jp/contests/abc{i:03}/tasks'
    driver.get(contest_url)
    rows = driver.find_elements_by_xpath('//table/tbody//tr')
    task_urls = []
    for row in rows:
        di['contest'].append(f'ABC{i:03}')
        di['url'].append(row.find_element_by_xpath('td[1]/a').get_attribute('href'))
        di['alphabet'].append(row.find_element_by_xpath('td[1]/a').text)
        di['task'].append(row.find_element_by_xpath('td[2]/a').text)
        
df = pd.DataFrame.from_dict(di)


client = TrelloClient(
    api_key=config.key_trello,
    api_secret=config.secret_trello,
    token=config.token_trello)

id_board = '5d627c63629fab5643938a84'
id_list = {'A': '5d7750ebb8fc04082063cc79',
           'B': '5d7750ec1fcb7236fcab556f',
           'C': '5d7750eca709b853c0d14602',
           'D': '5d7750ecd0e7f6368cfad8b7',
           'E': '5d7750ed11c927493f49d87d',
           'F': '5d7750edfe8cd27a2fea6592'}
labels = [label for label in board.get_labels() if label.name == 'todo']

board = client.get_board(id_board)
for alphabet in 'ABCDEF':
    list_ = board.get_list(id_list[alphabet])
    for i in df[df['alphabet']==alphabet].index:
        contest = df.loc[i, 'contest']
        url = df.loc[i, 'url']
        task = df.loc[i, 'task']
        list_.add_card(name=f'{contest} ({alphabet} - {task})',
                       desc=url,
                       labels=labels,
                       position='top')

C:\Users\taiki\Anaconda3\Lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.
